# Deep Learning Toolkit for Splunk - Simple Hidden Markov Model

This notebook contains a barebone example workflow how to work on custom containerized code that seamlessly interfaces with the Deep Learning Toolkit for Splunk.

Note: By default every time you save this notebook the cells are exported into a python module which is then invoked by Splunk MLTK commands like <code> | fit ... | apply ... | summary </code>. Please read the Model Development Guide in the Deep Learning Toolkit app for more information.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [186]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import numpy as np
import pandas as pd
import pomegranate as pg
from pomegranate import *
# ...
# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [187]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("pomegranate version: " + pg.__version__)

numpy version: 1.19.2
pandas version: 1.1.3
pomegranate version: 0.13.2


In [188]:
#import warnings
#warnings.filterwarnings('ignore')

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a dataset into your notebook environment. Note: mode=stage is used in the | fit command to do this.

index=_internal<br>
| table _time host punct<br>
| eventstats count as total_by_host by host<br>
| eventstats count as count_by_host by host punct<br>
| eval punct_freq_by_host=count_by_host/total_by_host<br>
| fit MLTKContainer mode=stage algo=hidden_markov_model punct into app:hidden_markov_model<br>


After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("barebone_model" in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [189]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [190]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
df, param = stage("hidden_markov_model")
print(param)
df

{'options': {'params': {'mode': 'stage', 'algo': 'hidden_markov_model'}, 'args': ['punct'], 'feature_variables': ['punct'], 'model_name': 'hidden_markov_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'disabled': False, 'handle_new_cat': 'default', 'max_distinct_cat_values': '10000', 'max_distinct_cat_values_for_classifiers': '10000', 'max_distinct_cat_values_for_scoring': '10000', 'max_fit_time': '6000', 'max_inputs': '100000000', 'max_memory_usage_mb': '10000', 'max_model_size_mb': '150', 'max_score_time': '6000', 'streaming_apply': '0', 'use_sampling': '1'}, 'kfold_cv': None}, 'feature_variables': ['punct']}


,punct
0,"--_::._+____-_=,_=""/////////."",_=.,_=.,_=.,_=,..."
1,"--_::._+____-_=,_=""/////////."",_=.,_=.,_=.,_=,..."
2,"--_::._+____-_=,_="""",_=.,_=.,_=.,_=,_=.,_="
3,"--_::._+____-_=,_="""",_=.,_=.,_=.,_=,_=.,_="
4,"--_::._+____-_=,_="""",_=.,_=.,_=.,_=,_=.,_="
...,...
6315,"--_::._+____-_=,_=,_=-,_=."
6316,"--_::._+____-_=,_=,_=-,_=."
6317,"--_::._+____-_=,_=,_=-,_=."
6318,"--_::._+____-_=,_=,_=-,_=."


## Stage 2 - create and initialize a model

In [191]:
# initialize your model
# available inputs: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    model = {}
    #model['hmm'] = HiddenMarkovModel("HMM")
    return model

In [192]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
model = init(df,param)
model

{}

## Stage 3 - fit the model

In [205]:
# train your model
# returns a fit info json object and may modify the model object
def fit(model,df,param):
    X = df[param['feature_variables'][0]]

    distinct_states = set(''.join(X.unique()))
    equal_probability = 1.0 / len(distinct_states)
    discreet_equal_states = { state : equal_probability for state in distinct_states }
    discreet_equal_states_distribution = DiscreteDistribution( discreet_equal_states )
    equal_state = State( discreet_equal_states_distribution, name="equal_state" )

    #model = {}
    hmm_model = HiddenMarkovModel("HMM")
    hmm_model.add_states( [equal_state] )
    hmm_model.add_transition( hmm_model.start, equal_state, 1.00 )
    hmm_model.add_transition( equal_state, equal_state, 0.99 )
    hmm_model.add_transition( equal_state, hmm_model.end, 0.01)
    hmm_model.bake()

    info = hmm_model.fit( [ list(x) for x in X ] , max_iterations=10, n_jobs=6 )
    model['hmm'] = hmm_model
    model['info'] = info
    #info = {"message": "model trained"}

    return info

In [206]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
info = fit(model,df,param)
print(info)

HMM:{
    "class" : "State",
    "distribution" : {
        "class" : "Distribution",
        "dtype" : "str",
        "name" : "DiscreteDistribution",
        "parameters" : [
            {
                ";" : 0.0005303121610220602,
                "=" : 0.15459563697722067,
                ">" : 0.0003615764734241291,
                "&" : 0.0016005785223575488,
                "'" : 0.00028926117873930533,
                "," : 0.11854887308665789,
                ":" : 0.06534892129685432,
                "\"" : 0.030435097023020677,
                "/" : 0.03593105941906945,
                "]" : 0.003991804266602511,
                "?" : 0.0010943714595637556,
                "<" : 0.0003615764734241291,
                "[" : 0.003991804266602509,
                "." : 0.08566469808364488,
                "+" : 0.029890321803061348,
                ")" : 0.0002892611787393033,
                "_" : 0.3580137399059901,
                "-" : 0.1065204290707485,
                "

## Stage 4 - apply the model

In [199]:
# apply your model
# returns the calculated results
def apply(model,df,param):
    field = param['feature_variables'][0]
    X = df[field]
    y_hat = X.apply(lambda x: model['hmm'].log_probability(list(x)))
    result = pd.DataFrame(y_hat).rename(columns={field: param['feature_variables'][0]+"_log_probability"})
    return result

In [200]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing or development purposes
res = apply(model,df,param)
res

,punct_log_probability
0,-113.704326
1,-113.704326
2,-83.958218
3,-83.958218
4,-83.958218
...,...
6315,-51.240068
6316,-51.240068
6317,-51.240068
6318,-51.240068


## Stage 5 - save the model

In [201]:
# save model to name in expected convention "<algo_name>_<model_name>"
def save(model,name):
    with open(MODEL_DIRECTORY + name + ".json", 'w') as file:
        file.write(model['hmm'].to_json())
    return model

In [202]:
save(model,"hmm_test")

{'hmm': {
     "class" : "HiddenMarkovModel",
     "name" : "HMM",
     "start" : {
         "class" : "State",
         "distribution" : null,
         "name" : "HMM-start",
         "weight" : 1.0
     },
     "end" : {
         "class" : "State",
         "distribution" : null,
         "name" : "HMM-end",
         "weight" : 1.0
     },
     "states" : [
         {
             "class" : "State",
             "distribution" : {
                 "class" : "Distribution",
                 "dtype" : "str",
                 "name" : "DiscreteDistribution",
                 "parameters" : [
                     {
                         ";" : 0.0005303121610220602,
                         "=" : 0.15459563697722067,
                         ">" : 0.0003615764734241291,
                         "&" : 0.0016005785223575488,
                         "'" : 0.00028926117873930533,
                         "," : 0.11854887308665789,
                         ":" : 0.06534892129685434,
       

## Stage 6 - load the model

In [203]:
# load model from name in expected convention "<algo_name>_<model_name>"
def load(name):
    model = {}
    with open(MODEL_DIRECTORY + name + ".json", 'r') as file:
        model_json = file.read()
        hmm = HiddenMarkovModel("HMM").from_json(model_json)
        model['hmm'] = hmm
    return model

In [204]:
saved_model = load("hmm_test")
saved_model

{'hmm': {
     "class" : "HiddenMarkovModel",
     "name" : "HMM",
     "start" : {
         "class" : "State",
         "distribution" : null,
         "name" : "HMM-start",
         "weight" : 1.0
     },
     "end" : {
         "class" : "State",
         "distribution" : null,
         "name" : "HMM-end",
         "weight" : 1.0
     },
     "states" : [
         {
             "class" : "State",
             "distribution" : {
                 "class" : "Distribution",
                 "dtype" : "str",
                 "name" : "DiscreteDistribution",
                 "parameters" : [
                     {
                         ";" : 0.0005303121610220602,
                         "=" : 0.15459563697722067,
                         ">" : 0.0003615764734241291,
                         "&" : 0.0016005785223575488,
                         "'" : 0.00028926117873930533,
                         "," : 0.11854887308665789,
                         ":" : 0.06534892129685434,
       

## Stage 7 - provide a summary of the model

In [213]:
# return a model summary
def summary(model=None):    
    returns = {"version": {"pomegranate": pg.__version__ } }
    if model!=None:
        if 'info' in model:
            returns['info'] = model['info']
        elif 'hmm' in model:
            returns['info'] = model['hmm'].to_json()
    return returns

In [214]:
summary(model)

{'version': {'pomegranate': '0.13.2'},
 'info': {
     "class" : "HiddenMarkovModel",
     "name" : "HMM",
     "start" : {
         "class" : "State",
         "distribution" : null,
         "name" : "HMM-start",
         "weight" : 1.0
     },
     "end" : {
         "class" : "State",
         "distribution" : null,
         "name" : "HMM-end",
         "weight" : 1.0
     },
     "states" : [
         {
             "class" : "State",
             "distribution" : {
                 "class" : "Distribution",
                 "dtype" : "str",
                 "name" : "DiscreteDistribution",
                 "parameters" : [
                     {
                         ";" : 0.0005303121610220602,
                         "=" : 0.15459563697722067,
                         ">" : 0.0003615764734241291,
                         "&" : 0.0016005785223575488,
                         "'" : 0.00028926117873930533,
                         "," : 0.11854887308665789,
                   

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code